In [1]:
import os
import mlflow
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import pandas as pd

%matplotlib inline

#alt.renderers.enable("html")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
tracking_uri = os.getenv("TRACKING_URI", default="http://localhost:5000")
mlflow.set_tracking_uri(tracking_uri)

## Experiment Parameter-Search TabNet

- Single random seed for model training for all models
- Single random seed for random split for all models
- Per run/model calculate atomic attribution using various methods (e.g. tabnet, integrated gradients, saliency, etc.)
- Single target objective (first) using BCE-loss
- Hyperparameter search using optuna tpe sampler (Tree-structured Parzen Estimator - https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html)

In [3]:
experiment_name = "herg_tn_opttpe1"
exp = mlflow.get_experiment_by_name(experiment_name)

runs = mlflow.search_runs(experiment_ids=exp.experiment_id)

In [4]:
#runs.iloc[1:].describe().T

In [5]:
#runs.iloc[1:].corr()

### Search space

In [4]:
search_space = [c for c in runs.columns if "search_space/" in c]
for r in runs[search_space].iloc[0].head(n=99):
    print(r)

{'name': 'decay_step', 'type': 'choice', 'values': [50, 200, 800]}
{'name': 'gamma', 'type': 'choice', 'values': [1.0, 1.2, 1.5]}
{'name': 'lr', 'type': 'choice', 'values': [0.04, 0.02, 0.01]}
{'name': 'lambda_sparse', 'type': 'choice', 'values': [0.0, 1e-06, 0.0001, 0.01]}
{'name': 'virtual_batch_size', 'type': 'choice', 'values': [16, 32, 64]}
{'name': 'decision_size', 'type': 'choice', 'values': [16, 24, 32, 64]}
{'name': 'decay_rate', 'type': 'choice', 'values': [0.8, 0.9, 0.95]}
{'name': 'momentum', 'type': 'choice', 'values': [0.1, 0.05, 0.02]}
{'name': 'nr_steps', 'type': 'choice', 'values': [3, 4, 5, 7]}


### Other + default parameters

(parameter in the search space are overwritten)

In [5]:
params = [c for c in runs.columns if "search_space" not in c and "params.args" in c]
for p, v in zip(params, runs[params].iloc[0].head(n=99)):
    print(f"{p.replace('params.args/', '')}: {v}")

scheduler_params: {'decay_step': 10, 'decay_rate': 0.95}
minimize: False
optimizer: adamw
attentive_type: sparsemax
momentum: 0.01
featurizer_chunksize: 100
alpha: 2.0
nr_steps: 4
patience: 10
normalize_input: True
sampler_name: tpe
experiment_name: herg_tn_opttpe1
checkpoint_minimize: True
relaxation_type: gamma_fixed
run_name: tpe
seed: 1073978726
max_steps: 1000
scheduler: exponential_decay
nr_layers: 4
pruner_name: None
cache_dir: ../../../data/herg/
log_sparsity: True
lr: 0.01
gamma: 1.5
featurizer_name: combined
lambda_sparse: 0.0
num_workers: 8
gradient_clip_val: 1.0
featurizer_kwargs: {'fold': 1024, 'radius': 3, 'return_count': True, 'use_chirality': True, 'use_features': True}
virtual_batch_size: 256
tracking_uri: http://localhost:5000
trials: 30
track_metrics: ...value too long for mlflow - not inserted
featurizer_mp_context: fork
split_size: (0.6, 0.2, 0.2)
objective_name: val/AUROC
patience_objective: val/loss
decision_size: 64
checkpoint_objective: val/loss
standardize: Fa

### Best parameters
- Best parameters found within this scenario for each component in the search space

In [6]:
params = [c for c in runs.columns if c and "params.best" in c]
for p, v in zip(params, runs[params].iloc[0].head(n=99)):
    print(f"{p.replace('params.args/', '')}: {v}")

params.best/decay_step: 800
params.best/momentum: 0.05
params.best/decision_size: 16
params.best/lr: 0.01
params.best/decay_rate: 0.9
params.best/gamma: 1.5
params.best/nr_steps: 3
params.best/lambda_sparse: 0.0001
params.best/virtual_batch_size: 64


### Metrics
- Include metrics for atomic attribution/weights
    - Values were calculated using active hergophores on all rows for which herg activity was predicted as negative
    - Threshold was calculated per individual model using threshold moving
    
- Attribution Methods:
    - Tabnet - using the aggregated masks of the architecture itself (self explaining)

In [7]:
metrics_names = [
    "metrics.test/sparsity_mask",
    "metrics.test/AUROC",
    "metrics.test/loss",
    "metrics.test/mean/avg_score_pred_inactive/tabnet"
    "metrics.test/mean/avg_score_pred_inactive/integrated_gradients",
    "metrics.test/mean/avg_score_pred_inactive/saliency",
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns].iloc[1:]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.describe().T.sort_values(by="mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
sparsity_mask,30.0,0.996008,0.001894,0.990547,0.996139,0.996411,0.996635,0.998967
AUROC,30.0,0.761266,0.105468,0.525076,0.688073,0.822147,0.833865,0.837871
integrated_gradients,30.0,0.621072,0.054105,0.518021,0.606275,0.622756,0.632444,0.726453
loss,30.0,0.573525,0.060860,0.507248,0.535560,0.537086,0.632710,0.691465
saliency,30.0,0.537266,0.028899,0.510379,0.522063,0.528773,0.537348,0.661744
tabnet,30.0,0.533026,0.058020,0.448488,0.483060,0.517186,0.579153,0.689873


#### Correlation

- Correlation between various metrics

In [10]:
metrics.corr(method="pearson")

,tabnet,sparsity_mask,saliency,AUROC,loss,integrated_gradients
tabnet,1.000000,-0.178403,0.336690,0.299701,-0.239761,0.296681
sparsity_mask,-0.178403,1.000000,-0.426184,0.482734,-0.498536,0.134538
saliency,0.336690,-0.426184,1.000000,-0.096785,0.124812,0.564019
AUROC,0.299701,0.482734,-0.096785,1.000000,-0.984970,0.350012
loss,-0.239761,-0.498536,0.124812,-0.984970,1.000000,-0.350423
integrated_gradients,0.296681,0.134538,0.564019,0.350012,-0.350423,1.000000


In [30]:
x = "AUROC"
y = "integrated_gradients"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
#chart.show()
chart

alt.LayerChart(...)

In [10]:
x = "AUROC"
y = "tabnet"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
#chart.show()
chart

alt.LayerChart(...)

In [8]:
x = "sparsity_mask"
y = "tabnet"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
#chart.show()
chart

alt.LayerChart(...)

In [9]:
x = "sparsity_mask"
y = "AUROC"

chart = alt.Chart(metrics).mark_point().encode(
    x=alt.X(x, scale=alt.Scale(zero=False)),
    y=alt.Y(y, scale=alt.Scale(zero=False))
)

chart = chart + chart.transform_regression(x, y).mark_line()
#chart.show()
chart


alt.LayerChart(...)

## Experiment Best MLP (kfold cross validated)

- Using best parameters found (see above)
    - Run 5-fold cross validation (was run 4 times)
    - Results/metrics were calculated using the mean of all individual runs (4 * 5 = 20 runs)
    - Per 5-fold cross validation (4 times) the seed for model training and splitting was chosen randomly

In [11]:
experiment_name = "herg_tn_best_kfold"
exp = mlflow.get_experiment_by_name(experiment_name)

query = "tags.mlflow.runName != 'summary'"
runs = mlflow.search_runs(experiment_ids=exp.experiment_id, filter_string=query)

### Seeds


In [12]:
runs[["run_id", "params.seed", "params.split_seed"]].head(n=99)

,run_id,params.seed,params.split_seed
0,51a039063ad14c0185c672417902e820,921898046,921898046
1,41c6cf1276a0423495040508f76c7435,921898046,921898046
2,03a38ebff501405fa0af1dac06404c9c,921898046,921898046
3,6d5c932f8ade44358c85d823c1376b27,921898046,921898046
4,0cabbea6c8484bbfba22d81d181ddd34,921898046,921898046
5,de797c4f36364d0eb690644878e3fa13,4119919135,4119919135
6,14617c84fefe47d6b67f3d5b22cfe1cf,4119919135,4119919135
7,bf14a218eb664ff9a4534a64b3407e93,4119919135,4119919135
8,377cfd34c33744d99ad52050a5ba3679,4119919135,4119919135
9,426142d03db44bffa3e80ccdb4f2fa98,4119919135,4119919135


### Parameters

In [13]:
params = [c for c in runs.columns if "search_space" not in c and "params" in c]
for p, v in zip(params, runs[params].iloc[0].head(n=99)):
    print(f"{p.replace('params.', '')}: {v}")

featurizer_mp_context: fork
patience_minimize: True
attribution_kwargs/label: active_g10
weights_summary: top
run_name: tn
smile2: c1ccccc1CCNC - active
check_val_every_n_epoch: 1
split_seed: 921898046
trainable_parameters: 279945
decision_size: 16
log_metrics: True
class_weights: None
featurizer_kwargs/radius: 3
smile0: CCOc1ccccc1 - active
num_sanity_val_batches: []
num_training_batches: 0
verbose_evaluate: True
limit_train_batches: 1.0
seed: 921898046
num_classes: 2
alpha: 2.0
deterministic: True
attribution_kwargs/threshold: 0.5357922315597534
num_test_batches: []
scheduler_params/decay_step: 800
num_sanity_val_steps: inf
optimizer_params/weight_decay: 0.0001
split_size: (5, 4, 0)
num_workers: 8
patience_objective: val/loss
auto_scale_batch_size: False
log_every_n_steps: 50
relaxation_type: gamma_fixed
fast_dev_run: False
categorical_size: None
max_steps: 1000
use_labels: ['active_g10']
attribution_kwargs/label_idx: 0
ignore_index: -100
checkpoint_objective: val/loss
scheduler_para

### Metrics

#### Performance on hERG dataset

In [14]:
metrics_names = [
    "metrics.test/AUROC",
    "metrics.test/loss",
    "metrics.test/sparsity_mask"
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.describe().T.sort_values(by="mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
sparsity_mask,20.0,0.996377,0.000364,0.995665,0.996120,0.996388,0.996595,0.997087
AUROC,20.0,0.825810,0.017265,0.793626,0.816415,0.825899,0.831454,0.859350
loss,20.0,0.531627,0.026284,0.495905,0.514613,0.528472,0.546957,0.593450


##### Discussion

- The average sparsity (over the test set) for all trained model is very high. From 2017 features only about 7-8 features are used per average!
- AUROC is lower compared to MLP or RF

#### Active Hergophores on Inactive Model Predictions

- Atomic attribution using various methods

In [18]:
metrics_names = [
    "metrics.test/mean/avg_score_pred_inactive/tabnet",
    "metrics.test/mean/avg_score_pred_inactive/integrated_gradients",
    "metrics.test/mean/avg_score_pred_inactive/saliency",
    "metrics.test/mean/avg_score_pred_inactive/saliency-absolute",
    "metrics.test/mean/avg_score_pred_inactive/input_x_gradient",
    "metrics.test/mean/avg_score_pred_inactive/occlusion",
    "metrics.test/mean/avg_score_pred_inactive/deeplift",
    "metrics.test/mean/avg_score_pred_inactive/shapley_value_sampling",
    "metrics.test/mean/avg_score_pred_inactive/noise_tunnel_ig",
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.describe().T.sort_values(by="mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
shapley_value_sampling,20.0,0.649415,0.069913,0.451552,0.626492,0.663796,0.692717,0.727581
integrated_gradients,20.0,0.637420,0.069589,0.465502,0.593297,0.649511,0.686302,0.721648
input_x_gradient,20.0,0.611016,0.041370,0.523053,0.596375,0.609460,0.636457,0.683403
occlusion,20.0,0.610158,0.045650,0.480663,0.597129,0.611852,0.636196,0.671489
saliency,20.0,0.534310,0.023690,0.493869,0.515819,0.534421,0.548638,0.588040
noise_tunnel_ig,20.0,0.520508,0.018934,0.488566,0.509283,0.516547,0.531430,0.567675
tabnet,20.0,0.485733,0.061296,0.429961,0.450613,0.464963,0.508357,0.685107
saliency-absolute,20.0,0.468421,0.024849,0.430491,0.455058,0.461126,0.475507,0.526042


##### Discussion

- Integrated gradients and Shapley Value Sampling performs best
- Variance is high among all methods 
- Tabnet Variance is high - including extrem performance/values almost similar to that of integrated gradients

#### Active Hergophores on Active Model Predictions

- Atomic attribution using various methods

In [19]:
metrics_names = [
    "metrics.test/mean/avg_score_pred_active/tabnet",
    "metrics.test/mean/avg_score_pred_active",
    "metrics.test/mean/avg_score_pred_active/integrated_gradients",
    "metrics.test/mean/avg_score_pred_active/saliency",
    "metrics.test/mean/avg_score_pred_active/saliency-absolute",
    "metrics.test/mean/avg_score_pred_active/input_x_gradient",
    "metrics.test/mean/avg_score_pred_active/occlusion",
    "metrics.test/mean/avg_score_pred_active/deeplift",
    "metrics.test/mean/avg_score_pred_active/shapley_value_sampling",
    "metrics.test/mean/avg_score_pred_active/noise_tunnel_ig",
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.describe().T.sort_values(by="mean", ascending=False)


,count,mean,std,min,25%,50%,75%,max
shapley_value_sampling,20.0,0.665873,0.040169,0.581195,0.641621,0.681724,0.690342,0.705551
integrated_gradients,20.0,0.661926,0.034200,0.589533,0.641616,0.659788,0.692895,0.710609
occlusion,20.0,0.634877,0.025842,0.573834,0.619782,0.629285,0.659211,0.672651
input_x_gradient,20.0,0.611718,0.035083,0.526670,0.598916,0.609057,0.640399,0.665167
saliency,20.0,0.536986,0.019177,0.495215,0.526984,0.536855,0.544501,0.578306
noise_tunnel_ig,20.0,0.513258,0.014065,0.486963,0.504614,0.513513,0.521729,0.538046
tabnet,20.0,0.488629,0.051813,0.406693,0.462644,0.473017,0.499786,0.622655
saliency-absolute,20.0,0.485707,0.024395,0.436811,0.474913,0.482293,0.495050,0.547129


#### Correlations

In [20]:
metrics_names = [
    "metrics.test/AUROC",
    "metrics.test/loss",
    "metrics.test/sparsity_mask"
]

metrics_names += [
    "metrics.test/mean/avg_score_pred_inactive/tabnet",
    "metrics.test/mean/avg_score_pred_inactive/integrated_gradients",
    "metrics.test/mean/avg_score_pred_inactive/saliency",
    "metrics.test/mean/avg_score_pred_inactive/saliency-absolute",
    "metrics.test/mean/avg_score_pred_inactive/input_x_gradient",
    "metrics.test/mean/avg_score_pred_inactive/occlusion",
    "metrics.test/mean/avg_score_pred_inactive/deeplift",
    "metrics.test/mean/avg_score_pred_inactive/shapley_value_sampling",
    "metrics.test/mean/avg_score_pred_inactive/noise_tunnel_ig",
]
metrics_columns = [c for c in runs.columns if any(c in m for m in metrics_names)]
metrics = runs[metrics_columns]

metrics.columns = [c.split("/")[-1] for c in metrics.columns]
metrics.corr(method="pearson")

,saliency-absolute,integrated_gradients,loss,tabnet,shapley_value_sampling,sparsity_mask,occlusion,noise_tunnel_ig,input_x_gradient,saliency,AUROC
saliency-absolute,1.000000,0.057381,-0.226265,0.673946,0.040885,0.027493,-0.006819,0.342307,0.195500,0.324990,0.222401
integrated_gradients,0.057381,1.000000,-0.346905,-0.140301,0.573344,-0.397321,0.288928,0.369849,0.342064,0.143196,0.284968
loss,-0.226265,-0.346905,1.000000,-0.144968,-0.001057,0.205727,0.152690,-0.305051,0.081962,0.216951,-0.792057
tabnet,0.673946,-0.140301,-0.144968,1.000000,-0.035143,0.007521,0.087076,0.099898,0.247996,0.249934,0.042475
shapley_value_sampling,0.040885,0.573344,-0.001057,-0.035143,1.000000,-0.175365,0.803070,0.123943,0.662979,0.367038,0.110549
sparsity_mask,0.027493,-0.397321,0.205727,0.007521,-0.175365,1.000000,-0.263553,-0.120901,-0.454737,-0.172071,-0.281738
occlusion,-0.006819,0.288928,0.152690,0.087076,0.803070,-0.263553,1.000000,0.066307,0.886380,0.643381,0.076303
noise_tunnel_ig,0.342307,0.369849,-0.305051,0.099898,0.123943,-0.120901,0.066307,1.000000,0.236170,0.182224,0.201025
input_x_gradient,0.195500,0.342064,0.081962,0.247996,0.662979,-0.454737,0.886380,0.236170,1.000000,0.752998,0.165183
saliency,0.324990,0.143196,0.216951,0.249934,0.367038,-0.172071,0.643381,0.182224,0.752998,1.000000,0.062805
